<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению
Автор материала: Мария Мансурова, аналитик-разработчик в команде Яндекс.Метрики 
    
Материал распространяется на условиях лицензии [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Можно использовать в любых целях (редактировать, поправлять и брать за основу), кроме коммерческих, но с обязательным упоминанием автора материала.

# <center> Домашнее задание № 9. (демо)
## <center> Анализ временных рядов
    
**Заполните пропущенный код и ответьте на вопросы в [онлайн-форме](https://docs.google.com/forms/d/1ijk4aFKY5plPiI8z3Mgi3i1Ln94VBY9SSt6xGIdVVFQ/).**

In [46]:
import pandas as pd
import os

from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import graph_objs as go
import requests
import numpy as np

print(__version__) # need 1.9.0 or greater

init_notebook_mode(connected = True)

def plotly_df(df, title = ''):
    data = []
    
    for column in df.columns:
        trace = go.Scatter(
            x = df.index,
            y = df[column],
            mode = 'lines',
            name = column
        )
        data.append(trace)
    
    layout = dict(title = title)
    fig = dict(data = data, layout = layout)
    iplot(fig, show_link=False)

4.0.0


## Подготавливаем данные

Для начала скачаем данные в `dataframe`. Сегодня будем предсказывать просмотры wiki-страницы [Machine Learning](https://en.wikipedia.org/wiki/Machine_learning). Данные я скачала с помощью библиотеки [Wikipediatrend](https://www.r-bloggers.com/using-wikipediatrend/) для `R`.

In [2]:
df = pd.read_csv('../../data/wiki_machine_learning.csv', sep = ' ')
df = df[df['count'] != 0]
df.head()

,date,count,lang,page,rank,month,title
81,2015-01-01,1414,en,Machine_learning,8708,201501,Machine_learning
80,2015-01-02,1920,en,Machine_learning,8708,201501,Machine_learning
79,2015-01-03,1338,en,Machine_learning,8708,201501,Machine_learning
78,2015-01-04,1404,en,Machine_learning,8708,201501,Machine_learning
77,2015-01-05,2264,en,Machine_learning,8708,201501,Machine_learning


In [3]:
df.shape

(383, 7)

In [5]:
df.date = pd.to_datetime(df.date)

In [6]:
plotly_df(df.set_index('date')[['count']])

## Предсказание с помощью Facebook Prophet

Для начала построим предсказание с помощью простой библиотеки `Facebook Prophet`. Для того, чтобы посмотреть на качество модели, отбросим из обучающей выборки последние 30 дней.

In [7]:
from fbprophet import Prophet

In [8]:
predictions = 30

df = df[['date', 'count']]
df.columns = ['ds', 'y']
train_df = df[:-predictions].copy()

In [10]:
%%time
## ВАШ КОД для построения модели ##
model = Prophet()
model.fit(train_df)

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


CPU times: user 69.4 ms, sys: 4.87 ms, total: 74.2 ms
Wall time: 181 ms


**Вопрос 1:** Какое предсказание числа просмотров wiki-страницы на 20 января? Ответ округлите до целого числа.

Оценим качество предсказания по последним 30 точкам.

In [15]:
## ВАШ КОД для оценки качества полученной модели ##
future = model.make_future_dataframe(periods=predictions)
future.tail()

,ds
378,2016-01-16
379,2016-01-17
380,2016-01-18
381,2016-01-19
382,2016-01-20


In [16]:
forecast = model.predict(future)
forecast.tail()

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,weekly_lower,weekly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
378,2016-01-16,2981.154024,1690.468516,2518.251310,2962.994111,3000.095166,-861.706169,-861.706169,-861.706169,-861.706169,-861.706169,-861.706169,0.0,0.0,0.0,2119.447855
379,2016-01-17,2986.577498,1846.524392,2643.169703,2967.373346,3006.710916,-720.728280,-720.728280,-720.728280,-720.728280,-720.728280,-720.728280,0.0,0.0,0.0,2265.849218
380,2016-01-18,2992.000971,2855.392779,3662.843965,2971.585448,3013.266388,281.332823,281.332823,281.332823,281.332823,281.332823,281.332823,0.0,0.0,0.0,3273.333794
381,2016-01-19,2997.424444,3125.500026,3936.612094,2976.092412,3019.588458,541.456429,541.456429,541.456429,541.456429,541.456429,541.456429,0.0,0.0,0.0,3538.880873
382,2016-01-20,3002.847917,3048.639977,3809.685520,2980.325152,3026.883084,425.567222,425.567222,425.567222,425.567222,425.567222,425.567222,0.0,0.0,0.0,3428.415139


**Вопрос 2**: Какое получилось MAPE?

**Вопрос 3**: Какое получилось MAE?

In [53]:
from sklearn.utils import check_array

def mean_absolute_percentage_error(y_true, y_pred): 
#     y_true, y_pred = check_array(y_true, y_pred)

    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [54]:
y_pred = forecast[-30:]['yhat']
y_true = df[-30:]['y']
mean_absolute_percentage_error(y_true.values, y_pred.values)

34.577147275595095

In [55]:
from sklearn.metrics import mean_absolute_error

In [56]:
mean_absolute_error(y_true, y_pred)

601.3806679529374

In [37]:
y_true

346    2469
345    2292
348    1671
347    1227
350    1367
349    1373
352    2181
351    2162
353    2107
354    1590
369    1159
370    1704
371    1724
365    3180
366    3180
367    3186
368    3055
373    2874
374    1674
380    1873
379    3083
378    3319
377    3145
384    3164
383    2743
382    1644
381    1836
376    2983
375    3389
372    3559
Name: y, dtype: int64

## Предсказываем с помощью ARIMA

In [68]:
%matplotlib inline
import matplotlib.pyplot as plt
import warnings
from scipy import stats
import statsmodels.api as sm

**Вопрос 4:** Проверим стационарность ряда с помощью критерия Дики-Фулера. Является ли ряд стационарным? Какое значение p-value?

In [63]:
## ВАШ КОД для проверки стационарности ряда ##
print("Критерий Дики-Фуллера: p=%f" % sm.tsa.stattools.adfuller(df['y'])[1])

Критерий Дики-Фуллера: p=0.015384


**Вопрос 5**: Далее перейдем к построению модели SARIMAX (`sm.tsa.statespace.SARIMAX`). Модель c какими параметрами лучшая по `AIC`-критерию?

In [69]:
## ВАШ КОД для построения модели ##
ps = range(0, 5)
d=1
qs = range(0, 4)
Ps = range(0, 5)
D=1
Qs = range(0, 1)

from itertools import product

parameters = product(ps, qs, Ps, Qs)
parameters_list = list(parameters)
len(parameters_list)

100

In [73]:
%%time
results = []
best_aic = float("inf")
warnings.filterwarnings('ignore')
from tqdm import tqdm
for param in tqdm(parameters_list):
    #try except нужен, потому что на некоторых наборах параметров модель не обучается
    try:
        model=sm.tsa.statespace.SARIMAX(train_df['y'], order=(param[0], d, param[1]), 
                                        seasonal_order=(param[3], D, param[3], 24)).fit(disp=-1)
    #выводим параметры, на которых модель не обучается и переходим к следующему набору
    except ValueError:
        #print('wrong parameters:', param)
        continue
    aic = model.aic
    #сохраняем лучшую модель, aic, параметры
    if aic < best_aic:
        best_model = model
        best_aic = aic
        best_param = param
    results.append([param, model.aic])
    
warnings.filterwarnings('default')

result_table = pd.DataFrame(results)
result_table.columns = ['parameters', 'aic']
result_table.sort_values(by = 'aic', ascending=True).head()

100%|██████████| 100/100 [01:36<00:00,  2.50s/it]

CPU times: user 4min 28s, sys: 47.9 s, total: 5min 16s
Wall time: 1min 36s


,parameters,aic
99,"(4, 3, 4, 0)",4982.046855
95,"(4, 3, 0, 0)",4982.046855
98,"(4, 3, 3, 0)",4982.046855
97,"(4, 3, 2, 0)",4982.046855
96,"(4, 3, 1, 0)",4982.046855


In [74]:
result_table.sort_values(by = 'aic', ascending=False).head()

,parameters,aic
0,"(0, 0, 0, 0)",5444.522608
2,"(0, 0, 2, 0)",5444.522608
3,"(0, 0, 3, 0)",5444.522608
4,"(0, 0, 4, 0)",5444.522608
1,"(0, 0, 1, 0)",5444.522608
